# Analiza czerniaka za pomocą fraktalnej sieci neuronowej

In [1]:
import numpy as np
from scipy.ndimage import measurements

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Sprawdzamy dostępne urządzenie

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Zapisujemy konfigurację do zmiennych.

In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

Definiujemy warstwę, która będzie tworzyła obraz z fraktalnych cech podanych jej obrazów.

In [4]:
class Fractal2D(tf.keras.layers.Layer):
    def __init__(self, kernel_size_range):
        super(Fractal2D, self).__init__()
        self.kernel_size_range = kernel_size_range

    def chessboard_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.reduce_max(
                    tf.math.abs(tf.math.subtract(patched_inputs, central_pixels)), 
                    axis=3), 
                kernel_size), 
            dtype=tf.int32)
    
    def euclidean_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.pow(
                    tf.math.reduce_sum(
                        tf.math.pow(
                            tf.math.subtract(patched_inputs, central_pixels), 
                            2), 
                        axis=3), 
                    0.5), 
                kernel_size), 
            dtype=tf.int32)
    
    def manhattan_distance(self, patched_inputs, central_pixels, kernel_size):
        return tf.cast(
            tf.math.less_equal(
                tf.math.reduce_sum(
                    tf.math.abs(tf.math.subtract(patched_inputs, central_pixels)), 
                    axis=3), 
                kernel_size), 
            dtype=tf.int32)
    
    def extract_binary_patches(self, inputs, kernel_size, distance_function):
        patched_inputs = tf.image.extract_patches(inputs,
                                                     sizes=(1, kernel_size, kernel_size, 1),
                                                     strides=(1, kernel_size, kernel_size, 1),
                                                     rates=(1, 1, 1, 1),
                                                     padding='SAME')
        _, rows, cols, _ = patched_inputs.shape
        patched_inputs = tf.reshape(patched_inputs, shape=(-1, kernel_size, kernel_size, 3))
        
        central_pixels = tf.image.resize_with_crop_or_pad(patched_inputs, 1, 1)
        
        return tf.reshape(distance_function(patched_inputs, central_pixels, kernel_size), shape=(-1, rows * cols, kernel_size, kernel_size))
    
    def calculate_probability_matrices(self, binary_inputs, kernel_size):
        number_of_ones = tf.map_fn(lambda binary_input: tf.map_fn(lambda binary_patch: tf.math.reduce_sum(binary_patch), 
                                                                  binary_input), 
                                   binary_inputs)
        _, patch_number = number_of_ones.shape
        return tf.math.bincount(number_of_ones,
                                minlength=1, 
                                maxlength=kernel_size ** 2 + 1, 
                                axis=-1) / patch_number
    
    def calculate_fractal_dimensions(self, probability_matrices):
        def fd_helper(matrix):
            return tf.math.reduce_sum(tf.math.divide(matrix, tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        return tf.map_fn(lambda matrix: fd_helper(matrix), probability_matrices)
    
    def calculate_lacunarity(self, probability_matrices):
        def m_helper(matrix):
            return tf.math.reduce_sum(tf.math.multiply(matrix, tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        
        def m2_helper(matrix):
            return tf.math.reduce_sum(tf.math.multiply(tf.math.pow(matrix, 2), tf.range(1, len(matrix) + 1, dtype=tf.float64)))
        
        return tf.map_fn(lambda probability_matrix: 
                         tf.math.divide(
                             tf.math.subtract(m2_helper(probability_matrix), 
                                               tf.math.pow(m_helper(probability_matrix), 2)), 
                             tf.math.pow(m_helper(probability_matrix), 2)), 
                         probability_matrices)
    
    def call(self, inputs):
        kernel_size_start, kernel_size_end = self.kernel_size_range
        
        for kernel_size in range(kernel_size_start, kernel_size_end + 1, 2):
            cd_binary_patches = self.extract_binary_patches(inputs, kernel_size, distance_function=self.chessboard_distance)
            cd_probability_matrices = self.calculate_probability_matrices(cd_binary_patches, kernel_size)
            cd_fractal_dimensions = self.calculate_fractal_dimensions(cd_probability_matrices)
            cd_lacunarity = self.calculate_lacunarity(cd_probability_matrices)
            print(f'lacunarity shape: {cd_lacunarity.shape}')
        return inputs

Ładujemy dane do trenowania i walidacji.

In [5]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0)
training_set = datagen.flow_from_directory('/small-data',
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           subset='training')
validation_set = datagen.flow_from_directory('/small-data',
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical',
                                             subset='validation')

Found 640 images belonging to 3 classes.
Found 159 images belonging to 3 classes.


Zapisujemy ilość rozpoznawalnych diagnoz.

In [6]:
DIAGNOSIS_NUMBER = len(training_set.class_indices)

Tworzymy model, który wykorzystuje wcześniej zdefiniowaną warstwę.

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    Fractal2D(kernel_size_range=(3, 41)),
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                   output_shape=[1280],
                   trainable=False),
    tf.keras.layers.Dense(DIAGNOSIS_NUMBER, activation='softmax')
])

lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(training_set, validation_data=validation_set, epochs=2)

Epoch 1/2
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)
lacunarity shape: (None,)


KeyboardInterrupt: 